**Data Science and AI for Energy Systems** 

Karlsruhe Institute of Technology

Institute of Automation and Applied Informatics

Summer Term 2024

---

# Exercise IX: Graph Neural Networks

**Remark:** Use the updated docker image with version 1.0.1 (or arm-1.0.1) from this exercise onwards

**Imports**

In [ ]:
import numpy as np

import torch 
from torch import nn, optim
from torch.nn import functional as F

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx, to_dense_adj
from torch_geometric.nn import GCNConv

from torchmetrics import R2Score

import networkx as nx

import matplotlib.pyplot as plt

## Problem IX.2 (programming) - Dynamic Stability Assessment of Power Grids using Graph Neural Networks

In this programming task, we look into the dynamic stability assessment of power grids using graph neural networks. Specifically, we are interested in predicting the single-node basin stability (SNBS). The SNBS quantifies the probability that the system recovers following a perturbation.
Here we use a dataset excerpt and code adapted from [Nauck et al.](https://pubs.aip.org/aip/cha/article/33/10/103103/2914062/Toward-dynamic-stability-assessment-of-power-grid). We want to predict the nodal SNBS using the adjacency matrix $A$ and the injected power $P$ per node as inputs. The prediction is purely based on the structure and topology of the grid. We compare results to a ground truth obtained by computationally intensive dynamical simulations.

***
**(a) Load the dataset**

In [ ]:
# dataset class inheriting from torch_geometric.data.Dataset
# the saved .pt files contain dataset instances of this class
class SNBSDataset(Dataset):
    def __init__(self):
        super(SNBSDataset, self).__init__()

    def len(self):
        return len(self.data)

    def get(self, index):
        return self.data[index]

You can load the datasets from the ```.pt``` files using ```torch.load()```

In [ ]:
train_set = torch.load('train_set.pt')
valid_set = torch.load('valid_set.pt')
test_set = torch.load('test_set.pt')

***
**(b) Pick a single sample from the training set. For this sample:**

In [ ]:
data = train_set[0]
data

**(i) Print the adjacency matrix of the graph.**

*hint: use ```to_dense_adj()``` from ```torch_geometric.utils``` to get the adjacency matrix from ```data.edge_index```*

In [ ]:
adj = to_dense_adj(data.edge_index)
adj_matrix = adj.squeeze(0)

print(adj_matrix)

**(ii) Visualize the graph and colour the nodes by their injected power.**

*hint: you can use ```to_networkx``` from ```torch_geometric.utils``` to convert the graph to a networkx graph and then plot via the draw function in networkx. The node features are accessible via the ```.x``` attribute of a single sample.*

In [ ]:
G = to_networkx(data, to_undirected=True)

In [ ]:
# Get the node feature for each node
node_labels = {i: f'{data.x[i, 0].item():.2f}' for i in range(data.num_nodes)}

# Plot the graph with node features as labels
plt.figure(figsize=(10, 10))
nx.draw(G, labels=node_labels, node_color=data.x[:, 0], cmap=plt.get_cmap('coolwarm'), node_size=500)
plt.show()

**(iii) Power grids are sparsely connected. According to [Schultz et al.](https://link.springer.com/article/10.1140/epjst/e2014-02279-6) their degree distribution has a local maximum at small degrees, an exponentially decaying tail, and a mean of around 2.8. Plot the degree distribution of this sample and compare it to these characteristics.**

*hint: use the ```.degree()``` attribute of the networkx graph from the previous task*

In [ ]:
# plot degree distribution
degrees = [val for (node, val) in G.degree()]
plt.hist(degrees, bins=20)
plt.xlabel('Degree')
plt.ylabel('# of nodes')
plt.show()

In [ ]:
average_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()

In [ ]:
average_degree

***
**(c) Complete the code for the GCN implementation in PyTorch geometric**

In [ ]:
# configuration dictionary
cfg = {}

# dataset batch sizes
cfg["train_set::batchsize"] = 19
cfg["test_set::batchsize"] = 500
cfg["valid_set::batchsize"] = 500

# model settings
cfg["num_layers"] = 3
cfg["num_channels"] = [1, 20, 20, 1]

cfg["activation"] = ["relu","relu","None"]

# training settings
cfg["manual_seed"] = 1
cfg["epochs"] = 400

# threshoold for evaluation accuracy
cfg["eval::threshold"] = .1

We start building the model with implementing the Graph Convolution:

In [ ]:
class GraphConvolutionModule(torch.nn.Module):
    def __init__(self, num_channels_in, num_channels_out, activation):
        super(GraphConvolutionModule, self).__init__()
        self.activation = activation
        self.conv = GCNConv(num_channels_in, num_channels_out, improved=False)

    def forward(self, data, x):
        x = self.conv(x, edge_index=data.edge_index)
        if self.activation == "relu":
            return F.relu(x)
        return x

Now we implement the Graph Convolutional Network as ```torch.nn.Module``` and use the Graph Convolution as building block:

In [ ]:
class GraphConvolutionNetwork(torch.nn.Module):
    def __init__(self, num_layers, num_channels, activation):
        super(GraphConvolutionNetwork, self).__init__()


        self.convlist = nn.ModuleList()
        for i in range(0, num_layers):
            num_c_in = num_channels[i]
            num_c_out = num_channels[i+1]
            conv = GraphConvolutionModule(
                num_channels_in=num_c_in, num_channels_out=num_c_out, activation=activation[i])
            self.convlist.append(conv)
        self.endSigmoid = nn.Sigmoid()

    def forward(self, data):
        x = data.x
        for i, _ in enumerate(self.convlist):
            x = self.convlist[i](data, x)
        x = self.endSigmoid(x)
        return x

***
**(d) Train a model and assess its performance**

In [ ]:
class GNNmodule(nn.Module):
    def __init__(self, config):
        super(GNNmodule, self).__init__()

        if torch.cuda.is_available():
            self.device = torch.device("cuda:0")
            self.cuda = True
            print("cuda availabe:: send model to GPU")
        else:
            self.cuda = False
            self.device = torch.device("cpu")
            print("cuda unavailable:: train model on cpu")

        # seeds
        torch.manual_seed(config["manual_seed"])
        torch.cuda.manual_seed(config["manual_seed"])
        np.random.seed(config["manual_seed"])

        if self.cuda:
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        model = GraphConvolutionNetwork(num_layers=config["num_layers"], num_channels=config["num_channels"], activation=config["activation"])

        model.to(self.device)

        self.model = model

        # criterion
        self.criterion = nn.MSELoss()
        self.criterion.to(self.device)

        # set optimizer
        self.optimizer = optim.SGD(model.parameters(), lr=0.3, momentum=.9, weight_decay=1e-9)

    def forward(self, x):
        # compute model prediction
        y = self.model(x)
        return y

    def train_epoch_regression(self, data_loader, threshold):
        self.model.train()
        loss = 0.
        correct = 0
        all_labels = torch.Tensor(0).to(self.device)
        all_predictions = torch.Tensor(0).to(self.device)
        for _, (batch) in enumerate(data_loader):
            batch.to(self.device)
            self.optimizer.zero_grad()
            output = torch.squeeze(self.model.forward(batch))
            labels = batch.y
            temp_loss = self.criterion(output, labels)
            temp_loss.backward()
            self.optimizer.step()
            correct += torch.sum((torch.abs(output - labels) < threshold))
            loss += temp_loss.item()
            all_labels = torch.cat([all_labels, labels])
            all_predictions = torch.cat([all_predictions, output])
        r2score = R2Score().to(self.device)
        R2 = r2score(all_predictions, all_labels)
        # accuracy
        accuracy = 100 * correct / all_labels.shape[0]
        return loss, accuracy.item(), R2.item()

    def eval_model_regression(self, data_loader, threshold):
        self.model.eval()
        with torch.no_grad():
            loss = 0.
            correct = 0
            all_labels = torch.Tensor(0).to(self.device)
            all_predictions = torch.Tensor(0).to(self.device)
            for batch in data_loader:
                batch.to(self.device)
                labels = batch.y
                output = torch.squeeze(self.model(batch))
                temp_loss = self.criterion(output, labels)
                loss += temp_loss.item()
                correct += torch.sum((torch.abs(output - labels) < threshold))
                all_predictions = torch.cat([all_predictions, output])
                all_labels = torch.cat([all_labels, labels])
            accuracy = 100 * correct / all_labels.shape[0]
        r2score = R2Score().to(self.device)
        R2 = r2score(all_predictions, all_labels)
        return loss, accuracy.item(), R2.item()

In [ ]:
# initialize model
gnnmodule = GNNmodule(cfg)


train_loader = DataLoader(
    train_set, batch_size=cfg["train_set::batchsize"], shuffle=True)
valid_loader = DataLoader(
    train_set, batch_size=cfg["valid_set::batchsize"], shuffle=False)
test_loader = DataLoader(
    test_set, batch_size=cfg["test_set::batchsize"], shuffle=False)

train_loss_all_epochs = []
train_accu_all_epochs = []
train_R2_all_epochs = []

test_loss_all_epochs = []
test_accu_all_epochs = []
test_R2_all_epochs = []

epochs = cfg["epochs"]
for epoch in range(1,epochs):
    print(f"Epoch {epoch}/{epochs}.. ")
    train_loss, train_accu, train_R2 = gnnmodule.train_epoch_regression(train_loader, cfg["eval::threshold"])
    train_loss_all_epochs.append(train_loss)
    train_accu_all_epochs.append(train_accu)
    train_R2_all_epochs.append(train_R2)
    test_loss, test_accu, test_R2 = gnnmodule.eval_model_regression(test_loader, cfg["eval::threshold"])
    test_loss_all_epochs.append(test_loss)
    test_accu_all_epochs.append(test_accu)
    test_R2_all_epochs.append(test_R2)
    print('train R2: ''{:3.2f}'.format(100 * train_R2) + '%')
    print('train accu: ''{:3.2f}'.format(train_accu) + '%')
    print('test R2: ''{:3.2f}'.format(100 * test_R2) + '%')
    print('test accu: ''{:3.2f}'.format(test_accu) + '%')
print("finished")